In [1]:
# Cell 1: Настройка путей, импорт и загрузка функций инференса
import os
import sys
import glob
import random

import torch
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

# Корень проекта — две папки выше текущего рабочего каталога
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Путь до папки custom_inference, чтобы Python видел yolo_inference
inference_dir = os.path.join(project_root, 'custom_inference')
if inference_dir not in sys.path:
    sys.path.insert(0, inference_dir)

# Импортируем наш модуль инференса для YOLO
import yolo_inference  # он должен лежать в ./custom_inference/yolo_inference.py

# Пути к моделям
MODEL_DIR = os.path.join(inference_dir, 'models', 'yolo')
model_path_1 = os.path.join(MODEL_DIR, 'best.pt')
model_path_2 = os.path.join(MODEL_DIR, 'yolo12m.pt')

# Пути к данным для оценки (датасет YOLO с images/ и labels/ + data.yaml)
YOLO_DATA_DIR = os.path.join(project_root, 'dataset', 'yolo')
IMAGES_DIR = os.path.join(YOLO_DATA_DIR, 'images/val')
LABELS_DIR = os.path.join(YOLO_DATA_DIR, 'labels/val')

print("Cell 1: Импорт выполнен, пути установлены. Ready to proceed.")


Cell 1: Импорт выполнен, пути установлены. Ready to proceed.


In [2]:
# Cell 2: Сбор списка изображений и их аннотаций, выбор случайного поднабора
# Соберём все файлы с расширением .jpg или .png в папке images
image_extensions = ('*.jpg', '*.jpeg', '*.png')
all_image_paths = []
for ext in image_extensions:
    all_image_paths.extend(glob.glob(os.path.join(IMAGES_DIR, ext)))
all_image_paths = sorted(all_image_paths)

# Проверим, что для каждого изображения существует соответствующий .txt в labels
def get_label_path(img_path):
    """
    По пути к изображению формирует путь до соответствующего .txt-файла с аннотациями.
    Предполагается, что структура папок images/… и labels/… совпадает, а расширение меняется на .txt.
    """
    rel = os.path.relpath(img_path, IMAGES_DIR)
    base, _ = os.path.splitext(rel)
    label_path = os.path.join(LABELS_DIR, base + '.txt')
    return label_path

# Отфильтруем только те изображения, у которых есть .txt
validated_image_paths = []
for img_path in all_image_paths:
    if os.path.isfile(get_label_path(img_path)):
        validated_image_paths.append(img_path)

# Если данных слишком много, выберем случайную подвыборку, например 500 изображений
random.seed(42)
sample_size = 500
if len(validated_image_paths) > sample_size:
    sampled_image_paths = random.sample(validated_image_paths, sample_size)
else:
    sampled_image_paths = validated_image_paths.copy()

print(f"Всего изображений с аннотациями: {len(validated_image_paths)}")
print(f"Будем использовать в сравнении: {len(sampled_image_paths)} (случайная подвыборка)")


Всего изображений с аннотациями: 1583
Будем использовать в сравнении: 500 (случайная подвыборка)


In [3]:
# Cell 3: Вспомогательные функции для чтения GT и вычисления IoU
def load_ground_truth(label_path, img_width, img_height):
    """
    Считывает YOLO-формат аннотаций из .txt и возвращает список GT-боксов в виде
    [class_id, x_min, y_min, x_max, y_max].
    """
    gt_boxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls_id = int(parts[0])
            x_center = float(parts[1]) * img_width
            y_center = float(parts[2]) * img_height
            w = float(parts[3]) * img_width
            h = float(parts[4]) * img_height
            x_min = x_center - w / 2
            y_min = y_center - h / 2
            x_max = x_center + w / 2
            y_max = y_center + h / 2
            gt_boxes.append([cls_id, x_min, y_min, x_max, y_max])
    return gt_boxes

def compute_iou(box1, box2):
    """
    box = [x_min, y_min, x_max, y_max]
    Возвращает IoU двух прямоугольников.
    """
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    if union_area <= 0:
        return 0.0
    return intersection_area / union_area


In [4]:
# Cell 4: Функции для метрик (Precision, Recall, AP, mAP на IoU=0.5)
def prepare_detection_lists(all_gt, all_preds, iou_threshold=0.5):
    """
    all_gt: dict: ключ = image_id, значение = список GT-боксов [class_id, x_min, y_min, x_max, y_max]
    all_preds: dict: ключ = image_id, значение = список предсказаний [class_id, confidence, x_min, y_min, x_max, y_max]

    Возвращает словари по классам:
    gt_by_class: {class_id: list of dict(image_id, bbox, used=False)}
    det_by_class: {class_id: list of dict(image_id, bbox, confidence)}
    """
    gt_by_class = {}
    det_by_class = {}

    # GT
    for img_id, gt_list in all_gt.items():
        for item in gt_list:
            cls_id, x_min, y_min, x_max, y_max = item
            if cls_id not in gt_by_class:
                gt_by_class[cls_id] = []
            gt_by_class[cls_id].append({'image_id': img_id,
                                        'bbox': [x_min, y_min, x_max, y_max],
                                        'used': False})

    # Предсказания
    for img_id, pred_list in all_preds.items():
        for item in pred_list:
            cls_id = item['class_id']
            conf = item['confidence']
            x_min, y_min, w, h = item['box']
            x_max = x_min + w
            y_max = y_min + h
            if cls_id not in det_by_class:
                det_by_class[cls_id] = []
            det_by_class[cls_id].append({'image_id': img_id,
                                         'bbox': [x_min, y_min, x_max, y_max],
                                         'confidence': conf})
    return gt_by_class, det_by_class

def calculate_ap_per_class(gt_entries, pred_entries, iou_threshold=0.5):
    """
    gt_entries: список словарей {'image_id', 'bbox', 'used': False}
    pred_entries: список словарей {'image_id', 'bbox', 'confidence'}
    Возвращает AP для одного класса.
    """
    # Сортируем предсказания по убыванию confidence
    pred_entries = sorted(pred_entries, key=lambda x: x['confidence'], reverse=True)
    tp = np.zeros(len(pred_entries))
    fp = np.zeros(len(pred_entries))

    n_positives = len(gt_entries)

    # Создадим словарь с GT по изображению для удобства
    gt_per_img = {}
    for idx, g in enumerate(gt_entries):
        img_id = g['image_id']
        if img_id not in gt_per_img:
            gt_per_img[img_id] = []
        gt_per_img[img_id].append({'bbox': g['bbox'], 'used': False})

    for idx, pred in enumerate(pred_entries):
        img_id = pred['image_id']
        pred_box = pred['bbox']
        max_iou = 0.0
        max_gt_idx = -1

        if img_id in gt_per_img:
            for gt_idx, gt in enumerate(gt_per_img[img_id]):
                if not gt['used']:
                    iou = compute_iou(pred_box, gt['bbox'])
                    if iou > max_iou:
                        max_iou = iou
                        max_gt_idx = gt_idx

        if max_iou >= iou_threshold and max_gt_idx >= 0:
            tp[idx] = 1
            gt_per_img[img_id][max_gt_idx]['used'] = True
        else:
            fp[idx] = 1

    # Накопительное суммирование
    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)
    recalls = cum_tp / (n_positives + 1e-6)
    precisions = cum_tp / (cum_tp + cum_fp + 1e-6)

    # Добавим точку (0,1) в начало
    recalls = np.concatenate(([0.0], recalls, [1.0]))
    precisions = np.concatenate(([1.0], precisions, [0.0]))

    # Коррекция precision: для точек справа налево
    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = max(precisions[i], precisions[i + 1])

    # Найдём индексы, где recall меняет значение
    idxs = np.where(recalls[1:] != recalls[:-1])[0]
    ap = 0.0
    for idx in idxs:
        ap += (recalls[idx + 1] - recalls[idx]) * precisions[idx + 1]
    return ap

def compute_map(all_gt, all_preds, iou_threshold=0.5):
    """
    all_gt, all_preds: словари с ключом = image_id, значением = список GT или предсказаний.
    Возвращает dict: {class_id: AP}, mAP (среднее по классам)
    """
    gt_by_class, det_by_class = prepare_detection_lists(all_gt, all_preds, iou_threshold)
    ap_per_class = {}
    for cls_id in gt_by_class.keys():
        gt_entries = gt_by_class.get(cls_id, [])
        pred_entries = det_by_class.get(cls_id, [])
        ap = calculate_ap_per_class(gt_entries, pred_entries, iou_threshold)
        ap_per_class[cls_id] = ap

    # Учтём классы, которые предсказываются, но отсутствуют в GT (AP = 0)
    for cls_id in det_by_class.keys():
        if cls_id not in ap_per_class:
            ap_per_class[cls_id] = 0.0

    # mAP
    if len(ap_per_class) > 0:
        mAP = np.mean(list(ap_per_class.values()))
    else:
        mAP = 0.0

    return ap_per_class, mAP


In [5]:
# Cell 5: Загрузка моделей
# Предполагаются следующие функции в yolo_inference:
#   load_model(path_to_weights) → возвращает объект модели
#   inference(model, image) → возвращает список детекций в формате:
#       [{'class_id': int, 'confidence': float, 'box': [x_min, y_min, width, height]}, ...]
model1 = yolo_inference.load_model(model_path_1)
model2 = yolo_inference.load_model(model_path_2)

print("Cell 5: Модели загружены:")
print(f"  Model 1: {model_path_1}")
print(f"  Model 2: {model_path_2}")


Cell 5: Модели загружены:
  Model 1: c:\Users\thjat\ml-system-design-aith\custom_inference\models\yolo\best.pt
  Model 2: c:\Users\thjat\ml-system-design-aith\custom_inference\models\yolo\yolo12m.pt


In [6]:
# Cell 6: Сбор GT и предсказаний на подвыборке
all_gt = {}      # ключ: image_id (строка пути), значение: список GT-боксов
all_preds_1 = {} # ключ: image_id, значение: список предсказаний от модели1
all_preds_2 = {} # аналогично для модели2

for img_path in sampled_image_paths:
    img_id = os.path.relpath(img_path, project_root)
    # Считываем изображение
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Загружаем GT
    label_path = get_label_path(img_path)
    gt_boxes = load_ground_truth(label_path, w, h)
    all_gt[img_id] = gt_boxes

    # Запускаем инференс первой модели
    preds1 = yolo_inference.inference(model1, img)
    all_preds_1[img_id] = preds1

    # Запускаем инференс второй модели
    preds2 = yolo_inference.inference(model2, img)
    all_preds_2[img_id] = preds2

print("Cell 6: Инференс на подвыборке выполнен.")
print(f"  Количество изображений: {len(all_gt)}")



0: 384x640 1 kilometer_post, 284.4ms
Speed: 2.3ms preprocess, 284.4ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 238.4ms
Speed: 1.1ms preprocess, 238.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 picket_post, 271.0ms
Speed: 1.8ms preprocess, 271.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 275.7ms
Speed: 1.6ms preprocess, 275.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 picket_post, 245.6ms
Speed: 2.3ms preprocess, 245.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 236.4ms
Speed: 1.4ms preprocess, 236.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 picket_post, 263.2ms
Speed: 1.2ms preprocess, 263.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 242.8ms
Speed: 1.2ms preprocess, 242.8m

In [7]:
# Cell 7: Вычисление метрик и сравнение
# mAP@0.5 для первой модели
ap_per_class_1, mAP_1 = compute_map(all_gt, all_preds_1, iou_threshold=0.5)
# mAP@0.5 для второй модели
ap_per_class_2, mAP_2 = compute_map(all_gt, all_preds_2, iou_threshold=0.5)

print("Cell 7: Результаты оценки:")
print(f"  Model 1 (best.pt) mAP@0.5 = {mAP_1:.4f}")
print(f"  Model 2 (yolo12m.pt) mAP@0.5 = {mAP_2:.4f}")

# Вывод AP по классам (опционально, если классов немного)
print("\nAP по классам (Model 1):")
for cls_id, ap in sorted(ap_per_class_1.items()):
    print(f"  Class {cls_id}: AP = {ap:.4f}")

print("\nAP по классам (Model 2):")
for cls_id, ap in sorted(ap_per_class_2.items()):
    print(f"  Class {cls_id}: AP = {ap:.4f}")

# Простой вывод: какая модель лучше по mAP
if mAP_1 > mAP_2:
    print("\nВывод: Model 1 (best.pt) показала лучшие результаты.")
elif mAP_2 > mAP_1:
    print("\nВывод: Model 2 (yolo12m.pt) показала лучшие результаты.")
else:
    print("\nВывод: Оба модели показали одинаковый mAP@0.5.")


Cell 7: Результаты оценки:
  Model 1 (best.pt) mAP@0.5 = 0.8554
  Model 2 (yolo12m.pt) mAP@0.5 = 0.0000

AP по классам (Model 1):
  Class 0: AP = 0.7877
  Class 1: AP = 0.9230

AP по классам (Model 2):
  Class 0: AP = 0.0000
  Class 1: AP = 0.0000
  Class 2: AP = 0.0000
  Class 6: AP = 0.0000
  Class 7: AP = 0.0000
  Class 8: AP = 0.0000
  Class 9: AP = 0.0000

Вывод: Model 1 (best.pt) показала лучшие результаты.


In [8]:
# Cell 8 (опционально): Сохранение результатов в DataFrame и CSV
results = {
    'class_id': [],
    'AP_model1': [],
    'AP_model2': []
}
all_classes = set(ap_per_class_1.keys()).union(set(ap_per_class_2.keys()))
for cls in sorted(all_classes):
    results['class_id'].append(cls)
    results['AP_model1'].append(ap_per_class_1.get(cls, 0.0))
    results['AP_model2'].append(ap_per_class_2.get(cls, 0.0))

df_results = pd.DataFrame(results)
df_results['diff_AP'] = df_results['AP_model2'] - df_results['AP_model1']

# Добавим строку с общим mAP
df_summary = pd.DataFrame([{
    'class_id': 'mAP@0.5',
    'AP_model1': mAP_1,
    'AP_model2': mAP_2,
    'diff_AP': mAP_2 - mAP_1
}])
df_out = pd.concat([df_results, df_summary], ignore_index=True)

# Сохраним в CSV рядом с ноутбуком
output_path = os.path.join(project_root, 'training', 'comparsion', 'yolo_comparison_results.csv')
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_out.to_csv(output_path, index=False)

print(f"Cell 8: Результаты сохранены в {output_path}")
df_out.head(10)


Cell 8: Результаты сохранены в c:\Users\thjat\ml-system-design-aith\training\comparsion\yolo_comparison_results.csv


,class_id,AP_model1,AP_model2,diff_AP
0,0,0.787745,0.0,-0.787745
1,1,0.923024,0.0,-0.923024
2,2,0.000000,0.0,0.000000
3,6,0.000000,0.0,0.000000
4,7,0.000000,0.0,0.000000
5,8,0.000000,0.0,0.000000
6,9,0.000000,0.0,0.000000
7,mAP@0.5,0.855385,0.0,-0.855385
